In [4]:
import torch
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM

In [ ]:
models = [
    "HuggingFaceTB/SmolLM-135M-Instruct",
    "meta-llama/Llama-3.2-3B-Instruct",
    "meta-llama/Llama-3.1-8B-Instruct",
]


In [ ]:
for model in models:
    model = AutoModelForCausalLM.from_pretrained(model, device_map="auto")
    number_of_total_parameters = sum(p.numel() for p in model.parameters())
    count_of_outliers = 0
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):
            try:
                sorted_weights, _ = torch.sort(module.weight.data.view(-1))
                k = int(0.25 * len(sorted_weights))
                lower_quartile = sorted_weights[k]
                upper_quartile = sorted_weights[-k]
                iqr = upper_quartile - lower_quartile
                lower_bound = lower_quartile - 1.5 * iqr
                upper_bound = upper_quartile + 1.5 * iqr
                outliers = module.weight.data[
                    (module.weight.data < lower_bound)
                    | (module.weight.data > upper_bound)
                ]
                count_of_outliers += len(outliers)
            except:
                print(name)

    print(f"{model}: Total number of outliers: {count_of_outliers}")
    print(f"{model}: Total number of parameters: {number_of_total_parameters}")
    print(
        f"{model}: Percentage of outliers: {count_of_outliers / number_of_total_parameters * 100:.2f}%"
    )

Total number of outliers: 1983917
Total number of parameters: 134515008
Percentage of outliers: 1.47%
